<a href="https://colab.research.google.com/github/raaraya1/Personal-Proyects/blob/main/Canales%20de%20Youtube/Python%20Engineer/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Random Forest**

Este algoritmo se contruye en base al algoritmo de **Decision Tree**. Asi, lo que se hace es:

- Definir cantidad de estimadores (**Decision Tree**)
- Cada estimador entrenarlo con una muestra del set de datos de entrenamiento, variando asi la cantidad de variables y la cantidad de datos con la cual se entrenan estos estimadores.
- Luego, para generar la prediccion de algoritmo, lo que se hace es consultar a cada estimador su prediccion y "**de manera democratica**" se escoje la opción mas "**votada**"

## **Construyendo el algoritmo desde cero**

In [6]:
# Importar bibliotecas
import numpy as np
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from collections import Counter

In [7]:
# A partir de esta funcion es que genermos los sets de entrenamiento particulares para cada arbol
def bootstrap_sample(X, y):
  n_samples = X.shape[0]
  idxs = np.random.choice(n_samples, size=n_samples, replace=True)
  return X[idxs], y[idxs]

# Con esta funcion buscamos extraer el valor (categoria) mas frecuente de una muestra
def most_common_label(y):
  counter = Counter(y)
  most_common = counter.most_common(1)[0][0]
  return most_common

class RandomForest:
  def __init__(self, n_trees=100, min_samples_split=2, max_depth=100, n_feats=None):
    self.n_trees = n_trees
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_feats = n_feats
    self.trees = []

  def fit(self, X, y):
    self.trees = []
    for _ in range(self.n_trees):
      # Generamos un arbol
      tree = DecisionTree(min_samples_split=self.min_samples_split,  
                          max_depth = self.max_depth, max_features=self.n_feats)
      
      # Generamos un set de entrenamiento particular para el arbol, lo entrenamos y lo guardamos en nuestra lista de estimadores
      X_sample, y_sample = bootstrap_sample(X, y)
      tree.fit(X_sample, y_sample) 
      self.trees.append(tree)


  def predict(self, X):
    # Generamos las predicciones 
    tree_preds = np.array([tree.predict(X) for tree in self.trees])

    # Cambiamos el esquema en como son guardados los datos
    tree_preds = np.swapaxes(tree_preds, 0, 1)

    # Se asigna la prediccion como el valor mas frecuente
    y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
    return np.array(y_pred)

## **Ahora probamos el algoritmo**

In [8]:
# Importamos las bibliotecas
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split


In [9]:
# Funcion para medir el desempeño del algoritmo
def accuracy(y_true, y_pred):
  accuracy = np.sum(y_true == y_pred) / len(y_true)
  return accuracy

# Importamos los datos
data = datasets.load_breast_cancer()
X = data.data
y = data.target

# Estos los separamos en los sets de entrenamiento y validacion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Cargamos el algortimo y lo entrenamos
clf = RandomForest(n_trees=3)
clf.fit(X_train, y_train)

# Generamos las predicciones y evaluamos su desempeño
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)
acc

0.9035087719298246

## **Ahora probemos el algortimo que ofrece sklearn**

In [1]:
# Importar el clasificador
from sklearn.ensemble import RandomForestClassifier as rf

In [5]:
# Llamamos al clasificador y lo entrenamos
clf_sk = rf(n_estimators=3)
clf_sk.fit(X_train, y_train)

# Generamos las prediccion y evaluamos el acierto del algoritmo
pred_sk = clf_sk.predict(X_test)
acc_sk = accuracy(y_test, pred_sk)
acc_sk

0.9035087719298246